In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold , cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder , StandardScaler , OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVR

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from sklearn.decomposition import PCA


In [2]:
data = pd.read_csv('file_version_6')

In [3]:
data.drop(['Unnamed: 0' , 'property_name','price_per_sqft','additionalRoom'] , inplace = True , axis = 1)

In [4]:
data.drop(np.where(data['society'].isnull())[0],inplace = True , axis = 0)

In [5]:
data['furnished_type'].value_counts()

furnished_type
0    1998
1     826
2     156
Name: count, dtype: int64

In [6]:
data['furnished_type'] = data['furnished_type'].replace({0:'unfurnished',1:'semifurnished',2:'fullfurnished'})

In [7]:
X = data.drop('price' , axis =1)
Y = data['price']

In [8]:
y_transformed = np.log1p(Y)

In [9]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2980 entries, 0 to 2980
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   society         2980 non-null   object 
 1   bedRoom         2980 non-null   int64  
 2   bathroom        2980 non-null   int64  
 3   balcony         2980 non-null   int64  
 4   floorNum        2980 non-null   float64
 5   agePossession   2980 non-null   object 
 6   built_up_area   2980 non-null   float64
 7   study room      2980 non-null   int64  
 8   pooja room      2980 non-null   int64  
 9   store room      2980 non-null   int64  
 10  others          2980 non-null   int64  
 11  servant room    2980 non-null   int64  
 12  furnished_type  2980 non-null   object 
 13  sector          2980 non-null   object 
dtypes: float64(2), int64(8), object(4)
memory usage: 349.2+ KB


In [10]:
object_columns = []
for i in X.columns:
    if X[i].dtype == 'O':
        object_columns.append(i)
    

In [11]:
object_columns

['society', 'agePossession', 'furnished_type', 'sector']

In [12]:
numeric_columns = []
for i in X.columns:
    if X[i].dtype != 'O':
        numeric_columns.append(i)
numeric_columns

['bedRoom',
 'bathroom',
 'balcony',
 'floorNum',
 'built_up_area',
 'study room',
 'pooja room',
 'store room',
 'others',
 'servant room']

In [13]:
data['society']

0       maa bhagwati residency
1                 apna enclave
2       tulsiani easy in homes
3          smart world orchard
4             parkwood westend
                 ...          
2976          ansal heights 86
2977     parsvnath green ville
2978           raheja vedaanta
2979           ambience lagoon
2980             dlf the crest
Name: society, Length: 2980, dtype: object

In [50]:
preprocessor = ColumnTransformer(transformers = [('numeric',StandardScaler() , numeric_columns),
                                                ('category' , OneHotEncoder(handle_unknown='ignore') , object_columns)],
                                remainder = 'passthrough')

In [51]:
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

In [52]:
 pipeline = Pipeline([('preprocessor',preprocessor),
                     ('xgboost',XGBRegressor())]
                    )

In [53]:
pipeline.fit(X,y_transformed)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('numeric', StandardScaler(),
                                                  ['bedRoom', 'bathroom',
                                                   'balcony', 'floorNum',
                                                   'built_up_area',
                                                   'study room', 'pooja room',
                                                   'store room', 'others',
                                                   'servant room']),
                                                 ('category',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['society', 'agePossession',
                                                   'furnished_type',
                                                   'sector'])])),
                ('xgb...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=None, n_jobs=None,
                              num_parallel_tree=None, random_state=None, ...))])

In [54]:
import pickle

with open('pipeline.pkl' , 'wb') as file:
    pickle.dump(pipeline , file)

In [49]:
with open('data.pkl' ,'wb') as file:
    pickle.dump(X , file)

In [40]:
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=5, scoring='r2')

In [41]:
scores.mean()

0.8080236167408898

In [24]:
from sklearn.ensemble import RandomForestRegressor

In [25]:
x_train , x_test , y_train ,y_test = train_test_split(X , y_transformed , test_size = 0.2 , random_state = 42)

In [26]:
pipeline.fit(x_train , y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('numeric', StandardScaler(),
                                                  ['bedRoom', 'bathroom',
                                                   'balcony', 'floorNum',
                                                   'built_up_area',
                                                   'study room', 'pooja room',
                                                   'store room', 'others',
                                                   'servant room']),
                                                 ('category',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['society', 'agePossession',
                                                   'furnished_type',
                                                   'sector'])])),
                ('random forest', RandomForestRegressor())])

In [27]:
y_pred = pipeline.predict(x_test)

In [28]:
y_pred = np.expm1(y_pred)

In [29]:
mean_absolute_error(np.expm1(y_test) ,y_pred)

0.3082346606352004

In [30]:
def scorer(model_name , model ):
    
    output = []
    output.append(model_name)
    
    pipeline = Pipeline([('preprocessor',preprocessor),
                        ('regressor' ,model)])
    
            # K-fold cross-validation
    kfold = KFold(n_splits = 10 , shuffle = True , random_state = 42)
    scores = cross_val_score(pipeline , X , y_transformed , cv = kfold , scoring = 'r2')
    
    output.append(scores.mean())
    
    x_train , x_test , y_train , y_test = train_test_split(X , y_transformed , test_size  = 0.2 , random_state = 42)
    
    pipeline.fit(x_train , y_train)
    
    y_pred = pipeline.predict(x_test)
    
    y_pred = np.expm1(y_pred)
    
    output.append(mean_absolute_error(np.expm1(y_test),y_pred))
    
    return output

In [31]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.ensemble import AdaBoostRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor

In [32]:
model_dict = {
    'linear_reg':LinearRegression(),
    'svr':SVR(),
    'decision tree': DecisionTreeRegressor(),
    'random forest':RandomForestRegressor(),
    'extra trees': ExtraTreesRegressor(),
    'gradient boosting': GradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'mlp': MLPRegressor(),
    'xgboost':XGBRegressor()
}

In [33]:
model_output = []
for model_name,model in model_dict.items():
    model_output.append(scorer(model_name, model))

In [34]:
model_output

[['linear_reg', 0.9023803418501881, 0.2503270125141867],
 ['svr', 0.8692490931992689, 0.3498124893927632],
 ['decision tree', 0.7868128785462101, 0.3417095967383612],
 ['random forest', 0.8567222070019007, 0.3108735340891417],
 ['extra trees', 0.8863676027829321, 0.2894997670137159],
 ['gradient boosting', 0.8325383998976743, 0.3841681196638644],
 ['adaboost', 0.6092392736424281, 0.6142950855960518],
 ['mlp', 0.91201160603304, 0.2551960386848329],
 ['xgboost', 0.8935594418671519, 0.28185624328435666]]

In [35]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.
